In [20]:
import os
from glob import glob
from os.path import join, exists
import nibabel as nib
import nilearn
import numpy as np

In [8]:
subj = 'S31'
date = '210817'

In [9]:
dir_root = join('/Users/clmn')
dir_data = join(dir_root, 'Desktop', 'new_preprocessed_data_from_SMC')

In [10]:
fwhm = 4 # Full width at half maximum
thresh_motion = 0.4

In [19]:
output_dir = join(dir_data,'test')
os.makedirs(output_dir, exist_ok=True)

In [26]:
epi = join(dir_data,'%s_%s_fMRI.nii'%(subj,date))
t1 = join(dir_data,'%s_%s_T1.nii'%(subj,date))

epi = nib.load(epi)
t1 = nib.load(t1)

epi.shape, t1.shape

((128, 128, 60, 300), (512, 512, 365))

In [24]:
epi.shape

(128, 128, 60, 300)

In [25]:
t1.shape

(512, 512, 365)